# Step 1b -- Creating scenario graphs 

This notebook presents various methods of preparing graphs for multiple investment / accessibility scenarios

In [1]:
import geopandas as gpd
import pandas as pd
import os, sys

# sys.path.append("../../../GOSTnets/GOSTnets")

import GOSTnets as gn
from GOSTnets.load_osm import *
import importlib
import networkx as nx
import osmnx as ox
from shapely import ops as ops
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely.geometry import LineString, MultiLineString, Point

### Setup

Define filepaths

In [4]:
input_pth = r'P:\BGD\GEO'
lcl_input_pth = r'inputs'
interm_pth = r'intermediate'
fin_pth = r'final'

Define parameters

In [5]:
simplif_meters = 25 # a bit of a manual process, higher number = simpler but more chance for errors
target_crs = 3106 # using Gulshan 303 / Bangladesh Traverse Mercator as it's a national metric projection with an established EPSG

In [6]:
# Production date for outputs being used

# prod_date = datetime.today().strftime('%y%m%d')
# prod_date = '210312'
prod_date = '210329'


### Scenario: Padma Bridge

Add in a new segment representing the Padma bridge, a major bridge investment in Bangladesh

In [7]:
G_Padma = nx.read_gpickle(os.path.join(fin_pth, f'final_current_G_{simplif_meters}m_{prod_date}.pickle'))

In [8]:
gn.example_edge(G_Padma)

(0, 24335, {'Wkt': 'LINESTRING (89.22178599999999 22.9686986, 89.2216605 22.9687017, 89.2211562 22.9686029, 89.22057150000001 22.9684942)', 'id': 349336, 'infra_type': 'unclassified', 'osm_id': '556994112', 'key': 'edge_349336', 'length': 126.88824810020613, 'Type': 'legitimate', 'unique_id': 0, 'time': 30.453179544049473, 'mode': 'drive'})


In [9]:
# identifying correct node for one side of the bridge by looking at the common point between two edges meeting at that node.
# This is necessary because the bride node has been renamed in the exported shapefiles due to some simplification. It appears that gostnets maintains the node integers in the background however
[u for u,v,d in G_Padma.edges(data = True) if d.get('key') == 'edge_43944']
[u for u,v,d in G_Padma.edges(data = True) if d.get('key') == 'edge_72435']

# 236053 it is

[68343, 236053]

In [10]:
G_Padma.get_edge_data(212524, 236053)

{0: {'Wkt': 'LINESTRING (90.26537821977453 23.4737997799302, 90.26579479999999 23.4775751)',
  'id': 43944,
  'infra_type': 'primary',
  'osm_id': '236821999',
  'key': 'edge_43944',
  'length': 379.1191237963699,
  'Type': 'origin_destruction',
  'unique_id': 535722,
  'time': 68.24144228334657,
  'mode': 'drive'}}

In [12]:
# Length manually calculated in QGIS
# assumes 20 km / h (5.555 m / second) speed.
# 6800 / 5.5555 = 1224
G_Padma.add_edge(205830,236053,\
                 Wkt = 'LINESTRING (90.25204 23.4051, 90.26014 23.41833, 90.2640 23.47380)', \
                 infra_type = 'trunk' ,\
                 key = 'edge_Padma', \
                 length = 6800, \
                 time = 1224, \
                 unique_id = 999999999,\
                 mode = 'drive')

'edge_Padma'

In [13]:
G_Padma

In [15]:
G_Padma.get_edge_data(205830,236053)

{'edge_Padma': {'Wkt': 'LINESTRING (90.25204 23.4051, 90.26014 23.41833, 90.2640 23.47380)',
  'infra_type': 'trunk',
  'length': 6800,
  'time': 1224,
  'unique_id': 999999999,
  'mode': 'drive'}}

In [19]:
??gn.save

Signature: gn.save(G, savename, wpath, pickle=True, edges=True, nodes=True)
Source:   
def save(G, savename, wpath, pickle = True, edges = True, nodes = True):
    """
    function used to save a graph object in a variety of handy formats

    :param G: a graph object
    :param savename: the filename, WITHOUT extension
    :param wpath: the write path for where the user wants the files saved
    :param pickle: if set to false, will not save a pickle of the graph
    :param edges: if set to false, will not save an edge gdf
    :param nodes: if set to false, will not save a node gdf
    """

    if nodes == True:
        new_node_gdf = node_gdf_from_graph(G)
        new_node_gdf.to_csv(os.path.join(wpath, '%s_nodes.csv' % savename))
    if edges == True:
        new_edge_gdf = edge_gdf_from_graph(G)
        new_edge_gdf.to_csv(os.path.join(wpath, '%s_edges.csv' % savename))
    if pickle == True:
        nx.write_gpickle(G, os.path.join(wpath, '%s.pickle' % savename))
File:      c:\user

In [20]:
gn.save(G_Padma, f'final_Padma_G_{simplif_meters}m_{prod_date}', 'final', nodes = False,  edges = True, pickle= False)

### Scenario: upgrade speeds on specific segments

Upgrade speeds on specific segments. Leftover, mildly cleaned code from a previous analysis in Cox's Bazaar

In [142]:
G_upgrade_all = nx.read_gpickle('final/final_current_G.pickle')
G_upgrade_noferry = nx.read_gpickle('final/final_current_G.pickle')

In [144]:
upgrade_segs = {'ferry' : ['85651714','85651729','43107027','409823924'], \
                'maheshkhali': ['236712168','423656274','424954822','424955827','424979570','425403567','468297576','468307000','468307004','468312439','468312440'], \
                'charkhari' : ['28887054','162212403','236712167','236712168','236712192','236712195','586014261']}

In [145]:
for u, v, data in G_upgrade_noferry.edges(data=True):
    if data['osm_id'] == '85651714':
        print(data['infra_type'])

ferry
ferry


In [146]:
for u, v, data in G_upgrade_noferry.edges(data = True):
    if data['osm_id'] in upgrade_segs['maheshkhali']:
        data['infra_type'] = 'secondary'
    elif data['osm_id'] in upgrade_segs['charkhari']:
        data['infra_type'] = 'primary'

for u, v, data in G_upgrade_all.edges(data = True):
    if data['osm_id'] in upgrade_segs['ferry']:
        data['infra_type'] = 'secondary'
    elif data['osm_id'] in upgrade_segs['maheshkhali']:
        data['infra_type'] = 'secondary'
    elif data['osm_id'] in upgrade_segs['charkhari']:
        data['infra_type'] = 'primary'
        

ferries are correctly not upgraded as long as they are loaded in as separate graphs

In [151]:
for u, v, data in G_upgrade_noferry.edges(data=True):
    if data['osm_id'] == '85651714':
        print(data['infra_type'])

ferry
ferry


In [149]:
# remove factor!! not needed because speeds were already coputed

G_upgrade_all_time = gn.convert_network_to_time(G_upgrade_all,
                                      distance_tag = 'length',
                                      graph_type = 'drive',
                                      road_col = 'infra_type',
                                      speed_dict = speeds
                                     )

G_upgrade_noferry_time = gn.convert_network_to_time(G_upgrade_noferry,
                                      distance_tag = 'length',
                                      graph_type = 'drive',
                                      road_col = 'infra_type',
                                      speed_dict = speeds
                                     )

In [150]:
gn.save(G_upgrade_all_time, 'final_upgrade_all_G', 'final', nodes = True,  edges = True)
gn.save(G_upgrade_noferry_time, 'final_upgrade_noferry_G', 'final', nodes = True,  edges = True)